# Training on Mobility Data
In this notebook we will train models on the mobility data.

In [ ]:
import os
import pandas as pd
from google.colab import auth
from datetime import datetime
auth.authenticate_user()
!gcloud source repos clone github_aistream-peelout_flow-forecast --project=gmap-997
os.chdir('/content/github_aistream-peelout_flow-forecast')
!pip install -r requirements.txt
!mkdir data
from flood_forecast.trainer import train_function
!pip install git+https://github.com/CoronaWhy/task-geo.git
!pip install git+https://github.com/coronawhy/task-ts
import wandb
!python corona_train.py --config da_config.json 
!gsutil cp gs://task_ts_data/acs2017_county_data.csv .

def format_pd(file_path="../../input/us-census-demographic-data/acs2015_census_tract_data.csv", output_path=""):
    df = pd.read_csv(file_path)
    df["full_county"] = df["County"] + "_" + df["State"]
    df.dropna().to_csv(file_path)

# Get config files and weights for auto-encoder
!wget -O "weight.pth" https://storage.googleapis.com/coronaviruspublicdata/experiments/09_September_202006_03AM_model.pth
!wget -O "config_right.json" https://storage.googleapis.com/coronaviruspublicdata/experiments/09_September_202006_03AM.json
!gsutil cp gs://task_ts_data/config_right.json . 
format_pd("acs2017_county_data.csv")

$ git clone https://github.com/AIStream-Peelout/flow-forecast
ERROR: (gcloud.source.repos.clone) Directory path specified exists and is not empty
mkdir: cannot create directory ‘data’: File exists


calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.


  Cloning https://github.com/CoronaWhy/task-geo.git to /tmp/pip-req-build-bwtb_9tl
  Running command git clone -q https://github.com/CoronaWhy/task-geo.git /tmp/pip-req-build-bwtb_9tl
     |████████████████████████████████| 71kB 1.7MB/s 
     |████████████████████████████████| 10.8MB 5.8MB/s 
     |████████████████████████████████| 61kB 4.8MB/s 
     |████████████████████████████████| 102kB 7.9MB/s 
     |████████████████████████████████| 51kB 4.0MB/s 
     |████████████████████████████████| 92kB 7.3MB/s 
     |████████████████████████████████| 2.6MB 35.0MB/s 
     |████████████████████████████████| 3.0MB 21.3MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 245kB 44.6MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
     |████████████████████████████████| 215kB 46.9MB/s 
     |████████████████████████████████| 215kB 46.6MB/s 
     |████████████████████████████████| 133kB 39.7MB/s 
     |████████████████████████████████| 245kB

python3: can't open file 'corona_train.py': [Errno 2] No such file or directory
Copying gs://task_ts_data/acs2017_county_data.csv...
- [1 files][607.5 KiB/607.5 KiB]                                                
Operation completed over 1 objects/607.5 KiB.                                    
--2020-10-21 06:02:56--  https://storage.googleapis.com/coronaviruspublicdata/experiments/09_September_202006_03AM_model.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 108.177.125.128, 74.125.23.128, 64.233.189.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|108.177.125.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168002 (164K) [application/octet-stream]
Saving to: ‘weight.pth’

weight.pth          100%[===================>] 164.06K  --.-KB/s    in 0.006s  

2020-10-21 06:02:56 (28.4 MB/s) - ‘weight.pth’ saved [168002/168002]

--2020-10-21 06:02:56--  https://storage.googleapis.com/coronaviruspublicdata/experiments/09_Sept

In [ ]:
!gsutil cp gs://task_ts_data/config_right.json . 
!wget -O "core_weights.pth" https://storage.googleapis.com/coronaviruspublicdata/experiments/18_October_202006_28AM_model.pth

Copying gs://task_ts_data/config_right.json...
/ [1 files][  1.5 KiB/  1.5 KiB]                                                
Operation completed over 1 objects/1.5 KiB.                                      
--2020-10-21 06:03:03--  https://storage.googleapis.com/coronaviruspublicdata/experiments/18_October_202006_28AM_model.pth
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.189.128, 108.177.97.128, 64.233.188.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.189.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 128767 (126K) [application/octet-stream]
Saving to: ‘core_weights.pth’

core_weights.pth    100%[===================>] 125.75K  --.-KB/s    in 0.002s  

2020-10-21 06:03:04 (59.1 MB/s) - ‘core_weights.pth’ saved [128767/128767]



Now that we have all the basic things setup lets define our config file. Since mobility data has a longer lag we will need to define our parameter sweeps to have a longer lookback window.

In [ ]:
def make_config_file(file_path, df_len, weight_path="core_weights.pth"):
  run = wandb.init(project="covid_forecast", entity="covid")
  wandb_config = wandb.config
  train_number = df_len*.6
  validation_number = df_len *.9
  number = wandb_config["forecast_history"]-1
  config_default={                 
    "model_name": "DARNN",
    "model_type": "PyTorch",
    "model_params": {
        "n_time_series":9,
        "forecast_history":wandb_config["forecast_history"],
        "hidden_size_encoder":wandb_config["hidden_encoder"],
        "decoder_hidden_size": wandb_config["hidden_decoder"],
        "out_feats": 1,
        "dropout": wandb_config["dropout"],
        "gru_lstm": False,
        "meta_data":{
            "method":"Bilinear2",
            "meta_dim":128,
            "params":{
                "in1_features":8,
                "in2_features":128,
                "out_features":8
            },
            "da_method":"down_sample45"
            
        }
    },
    "dataset_params":
    {  "class": "default",
       "training_path": file_path,
       "validation_path": file_path,
       "test_path": file_path,
       "forecast_test_len":14,
       "batch_size":wandb_config["batch_size"],
       "forecast_history":wandb_config["forecast_history"]-1,
       "forecast_length":wandb_config["out_seq_length"],
       "train_end": int(train_number),
       "valid_start":int(train_number+1),
       "valid_end": int(validation_number),
       "test_start":int(train_number+1),
       "target_col": ["rolling_7"],
       "relevant_cols": ["rolling_7", "month", "weekday", "mobility_retail_recreation",	"mobility_grocery_pharmacy",	"mobility_parks",	"mobility_transit_stations",	"mobility_workplaces",	"mobility_residential"],
       "scaler": "StandardScaler", 
       "interpolate": False
    },
    "training_params":
    {
       "criterion":"MSE",
       "optimizer": "Adam",
       "optim_params":
       {

       },
       "lr": wandb_config["lr"],
       "epochs": 10,
       "batch_size":wandb_config["batch_size"]
    
    },
    "meta_data": wandb_config["meta_data"],
    "GCS": True,
    "early_stopping":
    {
        "patience":3
    },
    "sweep":True,
    "wandb":False,
    "forward_params":{},
   "metrics":["RMSE"],
   "inference_params":
   {     
         "datetime_start":"2020-06-19",
          "hours_to_forecast":16, 
          "num_prediction_samples": 100,
          "test_csv_path":file_path,
          "decoder_params":{
              "decoder_function": "simple_decode", 
            "unsqueeze_dim": 1
          },
          "dataset_params":{
             "file_path": file_path,
             "forecast_history":wandb_config["forecast_history"]-1,
             "forecast_length":wandb_config["out_seq_length"],
             "relevant_cols": ["rolling_7", "month", "weekday", "mobility_retail_recreation",	"mobility_grocery_pharmacy",	"mobility_parks",	"mobility_transit_stations",	"mobility_workplaces",	"mobility_residential"],
             "target_col": ["rolling_7"],
             "scaling": "StandardScaler",
             "interpolate_param": False
          }
      }
  }
  if weight_path: 
    config_default["weight_path"] = weight_path
  wandb.config.update(config_default)
  print(config_default)
  return config_default

wandb_sweep_config_full = {
  "name": "Default sweep",
  "method": "grid",
  "parameters": {
        "batch_size": {
            "values": [10, 25, 30]
        },
        "lr":{
            "values":[0.001, 0.0001, .01, .1]
        },
        "forecast_history":{
            "values":[8]
        },
        "out_seq_length":{
            "values":[1]
        },
        "hidden_encoder":
        {
            "values":[32]
        },
        "hidden_decoder":
        {
            # 32, 64, 128 
            "values":[64]
        }, 
        "meta_data":{
            "values": [{
                "path":"config_right.json",
                "column_id": "full_county",
                "uuid": "Harris County_Texas",
                "dim":128,
                "input_method": "down_sample"
            }]
        },
        "dropout":
        {
            "values": [0.1, 0.5, 0.8]
        }
  
    }
}

In [ ]:
!wandb login
os.environ['MODEL_BUCKET'] = "coronaviruspublicdata"
os.environ["ENVIRONMENT_GCP"] = "Colab"
os.environ["GCP_PROJECT"] = "gmap-997"
# Flow Transfer Data 
!gsutil cp gs://predict_cfs/experiments/25_May_202010_29PM_model.pth .
# County Transfer Weights
!gsutil cp gs://coronaviruspublicdata/experiments/19_June_202011_00PM_model.pth .
# Flow + County Transfer Weights		
!gsutil cp gs://coronaviruspublicdata/experiments/22_June_202003_05PM_model.pth	.
!gsutil cp gs://coronaviruspublicdata/experiments/12_July_202011_43PM_model.pth .


wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
Copying gs://predict_cfs/experiments/25_May_202010_29PM_model.pth...
- [1 files][  4.7 MiB/  4.7 MiB]                                                
Operation completed over 1 objects/4.7 MiB.                                      
Copying gs://coronaviruspublicdata/experiments/19_June_202011_00PM_model.pth...
\ [1 files][  9.2 MiB/  9.2 MiB]                                                
Operation completed over 1 objects/9.2 MiB.                                      
Copying gs://coronaviruspublicdata/experiments/22_June_202003_05PM_model.pth...
\ [1 files][  9.2 MiB/  9.2 MiB]                                                
Operation completed over 1 objects/9.2 MiB.                                      
Copying gs://coronaviruspublicdata/experiments/12_July_202011_43PM_mode

In [ ]:
!rm -r data
!mkdir data
!gsutil cp -m -r gs://task_ts_data/2020-08-24/United_States__* data

CommandException: Incorrect option(s) specified. Usage:

  gsutil cp [OPTION]... src_url dst_url
  gsutil cp [OPTION]... src_url... dst_url
  gsutil cp [OPTION]... -I dst_url

For additional help run:
  gsutil help cp


In [ ]:
!gsutil cp gs://task_ts_data/2020-08-24/United_States__Texas__Harris_County.csv	.
!gsutil cp gs://task_ts_data/2020-08-24/United_States__Florida__Palm_Beach_County.csv .
!gsutil cp gs://task_ts_data/2020-08-24/United_States__Massachusetts__Middlesex_County.csv .
!gsutil cp gs://task_ts_data/2020-08-24/United_States__Kansas__Chase_County.csv .
!gustil cp gs://task_ts_data/2020-08-24/United_States__Kansas__Chase_County.csv .
!gsutil cp gs://task_ts_data/2020-08-24/United_States__New_York__Albany_County.csv .

Copying gs://task_ts_data/2020-08-24/United_States__Texas__Harris_County.csv...
/ [1 files][ 21.4 KiB/ 21.4 KiB]                                                
Operation completed over 1 objects/21.4 KiB.                                     
Copying gs://task_ts_data/2020-08-24/United_States__Florida__Palm_Beach_County.csv...
/ [1 files][ 22.2 KiB/ 22.2 KiB]                                                
Operation completed over 1 objects/22.2 KiB.                                     
Copying gs://task_ts_data/2020-08-24/United_States__Massachusetts__Middlesex_County.csv...
/ [1 files][ 27.1 KiB/ 27.1 KiB]                                                
Operation completed over 1 objects/27.1 KiB.                                     
CommandException: No URLs matched: gs://task_ts_data/2020-08-24/United_States__Kansas__Chase_County.csv
/bin/bash: gustil: command not found
Copying gs://task_ts_data/2020-08-24/United_States__New_York__Albany_County.csv...
- [1 files][ 20.8 KiB/ 20.8 Ki

## Loading the Data
In this section we will load our data using the data crawler.


In [ ]:
import glob
from corona_ts.data_utils.data_crawler import load_data
from corona_ts.data_utils.data_creator import loop_through_locations
from corona_ts.data_utils.data_creator import region_df_format
def make_df_from_csv(file_path):
  df = pd.read_csv(file_path)
  df['full_county'] = df['region'] + "_" + df['sub_region']
  df['country'] = df['country'].str.replace(" ","_")
  df['sub_region'] = df['country'] +"__"+df['region'].str.replace(" ", "_") + "__"+df['sub_region'].str.replace(" ", "_")
  return df 

In [ ]:
# Historical code to delete later
df_list_full_all_counties_total = [make_df_from_csv("United_States__Florida__Palm_Beach_County.csv"), make_df_from_csv("United_States__Florida__Palm_Beach_County.csv"), make_df_from_csv("United_States__Massachusetts__Middlesex_County.csv"), make_df_from_csv("United_States__Texas__Harris_County.csv")]
#df_list = loop_through_locations(df, minimum_datapoints_threshold=80)
#important_cities_list2 = ["United_States__California__San_Diego_County", "United_States__Florida__Broward_County", "United_States__Florida__Palm_Beach_County"]

In [ ]:
import pandas as pd
def get_most_recent_file(file_path):
  list_of_files = glob.glob(file_path+"/*.pth") # * means all if need specific format then *.csv
  if len(list_of_files) > 1:
    latest_file = max(list_of_files, key=os.path.getctime)
    return latest_file
  return None

def format_corona_data(region_df:pd.DataFrame, region_name:str):
  """
  Format data for a specific region into 
  a format that can be used with flow forecast. 
  """
  if region_name == 'county':
    region_name = region_df['full_county'].iloc[0]
  elif region_name=='state':
    region_name = region_df['state'].iloc[0]
  #else:
    #region_name = region_df['country'].iloc[0]
  d = pd.to_datetime(region_df['date'])
  region_df['date'] = pd.to_datetime(region_df['date'])
  region_df['month']=d.map(lambda x: x.month)
  region_df['weekday'] = d.map(lambda x: x.weekday())
  region_df['datetime'] = region_df.date
  region_df.index = region_df.date
  region_df = region_df.sort_index()
  region_df = region_df.fillna(0)
  region_df['new_cases'] = region_df['cases'].diff()
  region_df.iloc[0]['new_cases'] = 0
  region_df["rolling_7"] = region_df["new_cases"].rolling(7, win_type='triang').sum()/7.0
  region_df["diff_rolling"] = region_df["rolling_7"].diff()
  region_df= region_df.fillna(0)
  region_df.to_csv(region_name+".csv")
  print(region_df.head(9))
  return region_df, len(region_df), region_name+".csv"

def run_full_geo_code(df_list, start_index, end_index, use_transfer=True):
  for i in range(start_index, end_index):
    file_path, len_df, file_path_name = format_corona_data(df_list[i], 'county')
    print("Length of data frame is: ")
    print(len_df)
    latest_file = get_most_recent_file("model_save")
    sweep_full = wandb.sweep(wandb_sweep_config_transfer, project="covid_forecast", entity='covid')
    wandb.agent(sweep_full, lambda:train_function("PyTorch", make_config_file(file_path_name, len_df)), core_weights.pth)

def loop_special_counties(df):
  df['date'] = pd.to_datetime(df['date'])
  file_path, len_df, file_path2 = format_corona_data(df, "county")
  sweep_id = wandb.sweep(wandb_sweep_config_full,'covid', 'covid_forecast')
  wandb.agent(sweep_id, lambda: train_function("PyTorch", make_config_file(file_path2, len_df,  weight_path="core_weights.pth")))

for df in df_list_full_all_counties_total[3:]:
  loop_special_counties(df)
  break
#important_cities_list = ["United_States__California__Los_Angeles_County", "United_States__Illinois__Cook_County", "United_States__Arizona__Maricopa_County", "United_States__Massachusetts__Middlesex_County", "United_States__Texas__Dallas_County", "United_States__Texas__Harris_County", "United_States__Florida__Miami_Dade_County", "United_States__California__Riverside_County", "United_States__Colorado__Denver_County", "United_States__Ohio__Cuyahoga_County", "United_States__New_York__Queens_County", "United_States__New_York__Bronx_County", "United_States__New_York__New_York_County", "United_States__New_York__Richmond_County", "United_States__Michigan__Wayne_County", "United_States__Missouri__St._Louis_County"]
#important_cities_list3 = ["United_States__Maine__Cumberland_County", "United_States__Maine__York_County", "United_States__California__Los_Angeles_County", "United_States__Maine__Aroostook_County", "United_States__Maine__Androscoggin_County", "United_States__Maine__Franklin_County"]

#print(important_cities_list[14:])


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


            index       level  ... rolling_7 diff_rolling
date                           ...                       
2020-03-05   2956  sub_region  ...  0.000000     0.000000
2020-03-06   3210  sub_region  ...  0.000000     0.000000
2020-03-07   3557  sub_region  ...  0.000000     0.000000
2020-03-08   3665  sub_region  ...  0.000000     0.000000
2020-03-09   4395  sub_region  ...  0.000000     0.000000
2020-03-10   4756  sub_region  ...  0.000000     0.000000
2020-03-11   4942  sub_region  ...  0.000000     0.000000
2020-03-12   5727  sub_region  ...  0.321429     0.000000
2020-03-13   6129  sub_region  ...  0.357143     0.035714

[9 rows x 28 columns]
Create sweep with ID: s3y6lyoi
Sweep URL: https://wandb.ai/covid/covid_forecast/sweeps/s3y6lyoi


wandb: Agent Starting Run: e3ro4qiq with config:
wandb: 	batch_size: 10
wandb: 	dropout: 0.1
wandb: 	forecast_history: 8
wandb: 	hidden_decoder: 64
wandb: 	hidden_encoder: 32
wandb: 	lr: 0.001
wandb: 	meta_data: {'column_id': 'full_county', 'dim': 128, 'input_method': 'down_sample', 'path': 'config_right.json', 'uuid': 'Harris County_Texas'}
wandb: 	out_seq_length: 1
wandb: WARNING Ignored wandb.init() arg project when running a sweep
wandb: WARNING Ignored wandb.init() arg entity when running a sweep


DEBUG:google.auth._default:Checking /content/adc.json for explicit credentials as part of auth process...
DEBUG:google.auth._default:Checking /content/adc.json for explicit credentials as part of auth process...


{'model_name': 'DARNN', 'model_type': 'PyTorch', 'model_params': {'n_time_series': 9, 'forecast_history': 8, 'hidden_size_encoder': 32, 'decoder_hidden_size': 64, 'out_feats': 1, 'dropout': 0.1, 'gru_lstm': False, 'meta_data': {'method': 'Bilinear2', 'meta_dim': 128, 'params': {'in1_features': 8, 'in2_features': 128, 'out_features': 8}, 'da_method': 'down_sample45'}}, 'dataset_params': {'class': 'default', 'training_path': 'Texas_Harris_County.csv', 'validation_path': 'Texas_Harris_County.csv', 'test_path': 'Texas_Harris_County.csv', 'forecast_test_len': 14, 'batch_size': 10, 'forecast_history': 7, 'forecast_length': 1, 'train_end': 78, 'valid_start': 79, 'valid_end': 117, 'test_start': 79, 'target_col': ['rolling_7'], 'relevant_cols': ['rolling_7', 'month', 'weekday', 'mobility_retail_recreation', 'mobility_grocery_pharmacy', 'mobility_parks', 'mobility_transit_stations', 'mobility_workplaces', 'mobility_residential'], 'scaler': 'StandardScaler', 'interpolate': False}, 'training_param

...


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


plotting with CI now


epoch,9
validation_loss,0.1169
unscaled_validation_loss,60657.13714
_step,56
_runtime,13
_timestamp,1603265095
loss,0.02057
trg,553.57141
model_pred,510.49634
test_loss,0.09619
unscaled_test_loss,38957.75191


epoch,▁▁▂▂▃▃▃▃▄▄▅▅▆▆▆▆▇▇███
validation_loss,█▇▆▄▃▂▂▂▁▁
unscaled_validation_loss,█▇▆▄▃▂▂▂▁▁
_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
_runtime,▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▇█████████████████████
_timestamp,▁▁▁▁▁▁▁▁▁▁▁▁▂▂▂▂▂▂▇█████████████████████
loss,█▇▆▂▅▂▂▁▂█
trg,█▁▅▄
model_pred,█▁▇▄
test_loss,▁
unscaled_test_loss,▁


wandb: Sweep Agent: Waiting for job.
wandb: Sweep Agent: Exiting.


In [ ]:
!ls